- this notebook covers how to compute the logprobs / token probabilites and the perplexity when using langchain with huggingface
- as the return of the necessary outputs (i.e. logits) is not possible out of the box, some langchain and huggingface methods need to be overwritten

In [59]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline 
import os 
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from typing import Dict
from class_extender import ClassExtender
import time
from transformers import TextClassificationPipeline
from typing import Any, List, Optional

from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.outputs import Generation, LLMResult


os.environ["HF_TOKEN"]='your_huggingface_API_key'


BASE_MODEL_ID =  "/home/tpllmws23/llms/raftv2" # "mistralai/Mistral-7B-v0.3" # path to model to test

#"../../../llms/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.pad_token = tokenizer.unk_token

# Initialize language model
start_time = time.time()
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, torch_dtype=torch.bfloat16,
trust_remote_code=True, device_map="auto",
quantization_config=bnb_config)
load_time = time.time() - start_time

generation_config = GenerationConfig.from_pretrained(BASE_MODEL_ID)
generation_config.max_new_tokens = 1024 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.0001 # randomness of the generated tex
generation_config.top_p = 0 # diversity of the generated text
generation_config.do_sample = True 
generation_config.repetition_penalty = 1.2

#generation_config.use_cache=True,
#generation_config.num_return_sequences=1,

generation_config.output_logits=True,
generation_config.output_scores=True,
generation_config.output_hidden_states=True,
generation_config.return_dict_in_generate=True,
print(generation_config)


# we want to pass forward the input_ids and model_outputs (scores, logits, etc.) to evaluate them later
def postprocess(self, model_outputs, **postprocess_parameters: Dict):
    clean_up_tokenization_spaces=True
    input_ids = model_outputs["input_ids"]
    prompt_text = model_outputs["prompt_text"]
    generated_sequence = model_outputs["generated_sequence"][0]
    records = []
    for sequence in generated_sequence:
        # Decode text
        text = self.tokenizer.decode(
            sequence,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=clean_up_tokenization_spaces,
        )

        # Remove PADDING prompt of the sequence if XLNet or Transfo-XL model is used
        if input_ids is None:
            prompt_length = 0
        else:
            prompt_length = len(
                self.tokenizer.decode(
                    input_ids[0],
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=clean_up_tokenization_spaces,
                )
            )

        all_text = text[prompt_length:]
        if isinstance(prompt_text, str):
            all_text = prompt_text + all_text
            

        record = {"generated_text": all_text, "model_outputs": model_outputs["output"], "input_ids": input_ids}
    records.append(record)
    return records

# for decoderonlyoutput and setting return_dict=True and returning logits / scores the regular _forward method will fail (probably a bug in Huggingface transformers), as it returns a dict of outputs instead of the single generated sequence
def _forward(self, model_inputs, **generate_kwargs):
        input_ids = model_inputs["input_ids"]
        attention_mask = model_inputs.get("attention_mask", None)
        # Allow empty prompts
        if input_ids.shape[1] == 0:
            input_ids = None
            attention_mask = None
            in_b = 1
        else:
            in_b = input_ids.shape[0]
        prompt_text = model_inputs.pop("prompt_text")

        # If there is a prefix, we may need to adjust the generation length. Do so without permanently modifying
        # generate_kwargs, as some of the parameterization may come from the initialization of the pipeline.
        prefix_length = generate_kwargs.pop("prefix_length", 0)
        if prefix_length > 0:
            has_max_new_tokens = "max_new_tokens" in generate_kwargs or (
                "generation_config" in generate_kwargs
                and generate_kwargs["generation_config"].max_new_tokens is not None
            )
            if not has_max_new_tokens:
                generate_kwargs["max_length"] = generate_kwargs.get("max_length") or self.model.config.max_length
                generate_kwargs["max_length"] += prefix_length
            has_min_new_tokens = "min_new_tokens" in generate_kwargs or (
                "generation_config" in generate_kwargs
                and generate_kwargs["generation_config"].min_new_tokens is not None
            )
            if not has_min_new_tokens and "min_length" in generate_kwargs:
                generate_kwargs["min_length"] += prefix_length

        # BS x SL
        output = self.model.generate(input_ids=input_ids, attention_mask=attention_mask, **generate_kwargs)
        #print(generated_sequence)
        #warnings.warn(
        #            str(generated_sequence)
        #)
       
        generated_sequence = output.sequences[0]

        out_b = generated_sequence.shape[0]
        if self.framework == "pt":
            generated_sequence = generated_sequence.reshape(in_b, out_b // in_b, *generated_sequence.shape[1:])
        elif self.framework == "tf":
            generated_sequence = tf.reshape(generated_sequence, (in_b, out_b // in_b, *generated_sequence.shape[1:]))
        return {"generated_sequence": generated_sequence, "input_ids": input_ids, "prompt_text": prompt_text, "output": output}
        

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,
        generation_config=generation_config,
    )




with ClassExtender(type(pipe), postprocess), ClassExtender(TextClassificationPipeline, postprocess), ClassExtender(TextClassificationPipeline, _forward):

    llm = HuggingFacePipeline(pipeline=pipe)

    def create_eval_prompt(query: str, context: str):
        system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\n"
        return f"[INST]{system_prompt}Context: {context}\n\nQuestion: {query}[/INST]"
    
    data = [{"page_content": "\n\u00a7 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (\u00a7 59 LHG) sind f\u00fcr die in \u00a7 1 Abs. 1\nS. 1 genannten Studieng\u00e4nge deutsche Sprach kenntnisse nachzuweisen. 2Diese k\u00f6nnen\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundst\u00e4ndiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungspr\u00fcfung f\u00fcr ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachpr\u00fcfung f\u00fcr den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. \u201eTelc Deutsch C1 Hochschule\u201c\noder eine \u00e4quivalente Sprachpr\u00fcfung gem\u00e4\u00df der Rahmenordnung \u00fcber Deutsche\nSprachpr\u00fcfungen f\u00fcr das Studium an deutschen Hochschulen (RO -DT). 4Auf den Nachweis\neiner deutschen Sprachpr\u00fcfung kann bei Bewerber innen und Bewerbern im besonders\nbegr\u00fcndeten Einzelfall verzichtet werden, insbesondere wenn sie die deutsche\nStaatsangeh\u00f6rigkeit besitzen.\n(2) 1Sprachnachweise f\u00fcr den gew\u00e4hl ten Studiengang, die durch die Bewerberin oder den\nBewerber bis zum Bewerbungsschluss nicht vorgelegt werden k\u00f6nnen, k\u00f6nnen bis zum\nVorlesungsbeginn des Semesters gem\u00e4\u00df Terminplan der Hochschule Konstanz, f\u00fcr das  der\nAntrag auf Zulassung gestellt wurde, nachgereicht werden. 2Die Zulassung erfolgt in diesem\nFall gem \u00e4\u00df \u00a7 6  Abs. 5  unter Vorbehalt .\n(3) 1F\u00fcr Zeitstudierende gelten die Regelungen in \u00a7 10 Zulassungs - und\nImmatrikulationsordnung (ZIO) der Hochschule Konstanz.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21b Mechatronik (MME) Berufsbegleitendes Studium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Systemtechnik,\nMaschinenbau, Elektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer\nverwandten Fachrichtung.\n2. Englischkenntnisse, \u00e4quivalent z u Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote.  Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1 Satz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutsc her Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messtechnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teil note 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21a Mechatronik (MME) Vollzeitstudium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Maschinenbau,\nElektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer verwandten\nFachrichtung.\n2. Englischkenntnisse, \u00e4quivalent zu Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klass e) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote. Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1\nSatz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutscher Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messt echnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9 Zugangs - und Auswahlkriterien in den Masterstudieng\u00e4ngen\n(1)  1Im Besonderen Teil (\u00a7\u00a7 12 -26) dieser Satzung k\u00f6nnen ein oder mehrere der in Absatz 2\ngenannten Auswahlkriterien als weitere Zugangskriterien festgelegt werden. 2N\u00e4heres\nregelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7 12 -26).\n(2)  1F\u00fcr die Bildung der Ranglisten f\u00fcr das erste Fachsemester in den Masterstudieng\u00e4ngen\nwird, neben dem Ergebnis des fachlich einschl\u00e4gigen Hochschulabschlusses oder des\ngleichwertigen Abschlusses,  mindestens eines der folgenden  Auswahlkriterien\nber\u00fccksichtigt:\n1. Leistungen, die in dem Studium erbracht wurden, das Voraussetzung f\u00fcr den Zugang\nzu dem Masterstudiengang ist ,   Seite 8 von 43 2. Englischkenntnisse , n\u00e4heres regelt der Besondere Teil f\u00fcr den jeweiligen Studiengang\n(\u00a7\u00a7 12 -26),\n3. Berufst\u00e4tigkeit und Qualifikationen:\na) Art einer abgeschlossenen Berufsausbildung oder einer Berufst\u00e4tigkeit in einem\nanerkannten Ausbildungsberuf  oder eine andere einschl\u00e4gige Berufst\u00e4tigkeit , die \u00fcber\ndie fachspezifische Eignung Auskunft gibt, jeweils  einzeln und in Kombination, und\nb) Qualifikation en, die \u00fcber die fachspezifische Leistung Auskunft geben, jeweils einzeln\noder in Kombination,\n4. das Ergebnis eines fachspezifischen Studieneignungstests ,\n5. das Ergebnis des Auswahlgespr\u00e4chs/anderen m\u00fcndlichen Verfahrens  gem\u00e4\u00df \u00a7 9a ,\n6. ein Motivationsschreiben,\n7. eine schriftliche Abhandlung (Essay).\n2N\u00e4heres sowie die Gewichtung regelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7\n12-26).\n(2) 1Die Auswahl f\u00fcr h\u00f6here Fachsemester erfolgt gem\u00e4\u00df \u00a7 7 HZG i. V. m. \u00a7 32 HZVO.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
    context = [entry['page_content'] for entry in data]
    #eval_prompt = create_eval_prompt("""Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?#""", str(context))
    eval_prompt = create_eval_prompt("""Was sind Zugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik?""", str(context))
    
    # overwrite langchains _generate method to pass forward the model outputs etc.
    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        # List to hold all results
        text_generations: List[str] = []
        model_outputs: List[Any] = []
        pipeline_kwargs = kwargs.get("pipeline_kwargs", {})

        for i in range(0, len(prompts), self.batch_size):
            batch_prompts = prompts[i : i + self.batch_size]

            # Process batch of prompts
            responses = self.pipeline(
                batch_prompts,
                **pipeline_kwargs,
            )

            # Process each response in the batch
            for j, response in enumerate(responses):
                if isinstance(response, list):
                    # if model returns multiple generations, pick the top one
                    response = response[0]

                if self.pipeline.task == "text-generation":
                    # pass forward model outputs
                    text = response["generated_text"]
                    model_output = {
                        "logits": response["model_outputs"]["logits"],
                        "sequences": response["model_outputs"]["sequences"],
                        "input_ids": response["input_ids"],
                        "scores": response["model_outputs"]["scores"],
                    }
                elif self.pipeline.task == "text2text-generation":
                    text = response["generated_text"]
                elif self.pipeline.task == "summarization":
                    text = response["summary_text"]
                elif self.pipeline.task in "translation":
                    text = response["translation_text"]
                else:
                    raise ValueError(
                        f"Got invalid task {self.pipeline.task}, "
                        f"currently only {VALID_TASKS} are supported"
                    )

                # Append the processed text to results
                text_generations.append(text)
                if model_output is not None: model_outputs.append(model_output)

        return LLMResult(
            generations=[[Generation(text=text)] for text in text_generations],
            llm_output={
                "model_outputs": model_outputs
            }
        )
    
    def invoke(self, input: str) -> LLMResult:
        return (
            self.generate_prompt(
                [self._convert_input(input)],
            )
        )
    

    start_time = time.time()
    input_ids = tokenizer(eval_prompt, return_tensors="pt").input_ids
    with torch.no_grad():
        output = model(input_ids)
    prompt_eval_time = time.time() - start_time


    output: LLMResult = LLMResult(generations=[], llm_output={})
    
    start_time = time.time()
    with ClassExtender(type(llm), invoke), ClassExtender(type(llm), _generate):
        output = llm.invoke(eval_prompt) # type: ignore
    eval_time = time.time() - start_time


    

    if output.llm_output is None:
        print("MISSING LLM Outpout") 
        #return
    print(output.llm_output["model_outputs"])
    logits = output.llm_output["model_outputs"][0]["logits"]
    sequences = output.llm_output["model_outputs"][0]["sequences"]
    input_ids = output.llm_output["model_outputs"][0]["input_ids"]
    scores = output.llm_output["model_outputs"][0]["scores"]

    num_prompt_tokens = len(input_ids[0])
    num_generated_tokens = len(sequences[-1]) - num_prompt_tokens
    total_tokens = num_prompt_tokens + num_generated_tokens
    tokens_per_second = total_tokens / (prompt_eval_time + eval_time)

    # simulate llama.cpp timings (without sample time)
    print(f"Load time: {load_time * 1000:.2f} ms")
    print(f"Prompt eval time: {prompt_eval_time * 1000:.2f} ms / {num_prompt_tokens} tokens "
        f"({prompt_eval_time / num_prompt_tokens * 1000:.2f} ms per token, "
        f"{num_prompt_tokens / prompt_eval_time:.2f} tokens per second)")
    print(f"Eval time: {eval_time * 1000:.2f} ms / {num_generated_tokens} tokens "
        f"({eval_time / num_generated_tokens * 1000:.2f} ms per token, "
        f"{num_generated_tokens / eval_time:.2f} tokens per second)")
    print(f"Total time: {(prompt_eval_time + eval_time) * 1000:.2f} ms / {total_tokens} tokens")

    

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.01s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1024,
  "output_hidden_states": [
    true
  ],
  "output_logits": [
    true
  ],
  "output_scores": [
    true
  ],
  "repetition_penalty": 1.2,
  "return_dict_in_generate": [
    true
  ],
  "temperature": 0.0001,
  "top_p": 0
}

[{'logits': (tensor([[-422.0000,    0.4844,    2.5625,  ...,   -5.5000,   -4.0000,
           -3.5625]]), tensor([[-302.0000,   -6.4062,   -0.3145,  ...,   -5.9688,   -4.7188,
           -6.5312]]), tensor([[-239.0000,   -8.5000,    3.4531,  ...,   -7.2812,   -8.3125,
           -6.4062]]), tensor([[-258.0000,   -8.3125,    4.4688,  ...,   -7.0625,   -7.7188,
           -7.4688]]), tensor([[-274.0000,   -7.5312,    2.7344,  ...,   -7.6875,   -7.5625,
           -7.2500]]), tensor([[-298.0000,   -8.1250,    3.2031,  ...,   -6.8438,   -8.0625,
           -7.5625]]), tensor([[-213.0000,   -9.5625,    3.4688,  ...,   -6.5312,   -8.6875,
           -7.6250]]), t

In [56]:
output.generations[0][0].text # this will only return the prompt, the true generation lies inside sequences[0]

"[INST]You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\nContext: ['\\n§ 4 Sprachkenntnisse\\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (§ 59 LHG) sind für die in § 1 Abs. 1\\nS. 1 genannten Studiengänge deutsche Sprach kenntnisse nachzuweisen. 2Diese können\\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\\ngrundständiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\\n1. Feststellungsprüfung für ein Bachelorstudium durch Vorlage der Zugangsberechtigung\\ndes Studienkollegs an der Hochschule Konstanz,\\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachprüfung für den Hochschulzugang (DSH), sofern die DSH mit\\nmindestens der Stufe DSH -2 abges

In [61]:
import torch
import torch.nn.functional as F


# Assuming `logits` is a tensor of shape (sequence_length, vocab_size)
log_probs = F.log_softmax(torch.stack(list(logits), dim=0).squeeze(), dim=-1)

nll = F.nll_loss(log_probs, sequences[-1][input_ids.shape[1]:], reduction='none') # mask away prompt
average_nll = nll.mean()
# Perplexity is the exponentiation of the average NLL
perplexity = torch.exp(average_nll)
print("PERPLEXITY: ", perplexity)

print("\n\nLOGPROBS: ", log_probs)

def score_probs(log_probs, sequence_ids):
    sequence_tokens = [tokenizer.decode(id) for id in sequence_ids]
    token_logprobs = []
    for k in range(1, sequence_ids.shape[0]):
        token_logprobs.append(log_probs[k-1, sequence_ids[k]])
    return sequence_tokens, token_logprobs

# combine log_probs with tokens
result = score_probs(log_probs, sequences[-1][input_ids.shape[1]:] )

sequence_ids = sequences[-1][input_ids.shape[1]:]

sequence_tokens = [tokenizer.decode(id) for id in sequence_ids]

print("SCORE: ", result)

print("NLL: ", nll)



PERPLEXITY:  tensor(0.7297)


LOGPROBS:  tensor([[-4.3830e+02, -1.5816e+01, -1.3738e+01,  ..., -2.1800e+01,
         -2.0300e+01, -1.9863e+01],
        [-3.2001e+02, -2.4421e+01, -1.8329e+01,  ..., -2.3983e+01,
         -2.2733e+01, -2.4546e+01],
        [-2.6047e+02, -2.9974e+01, -1.8020e+01,  ..., -2.8755e+01,
         -2.9786e+01, -2.7880e+01],
        ...,
        [-2.1625e+02, -3.3063e+01, -1.1688e+01,  ..., -3.2126e+01,
         -3.0876e+01, -2.9876e+01],
        [-2.7100e+02, -2.8563e+01, -7.3759e+00,  ..., -3.0813e+01,
         -2.9876e+01, -2.7001e+01],
        [-5.1752e+02, -2.2567e+01, -1.9638e-02,  ..., -2.4957e+01,
         -2.5363e+01, -2.5395e+01]])
SCORE:  (['To', 'answer', 'this', 'question', ',', 'we', 'need', 'to', 'identify', 'the', 'requirements', 'or', 'pr', 'ere', 'quis', 'ites', 'mentioned', 'in', 'the', 'context', 'for', 'admission', 'into', 'the', 'Master', "'", 's', 'program', 'in', 'Me', 'chat', 'ron', 'ics', '.', 'The', 'relevant', 'information', 'can', 'be

In [44]:

vocab_size = 32768
# scores and logits need to be stacked as they are returned as a tuple
torch.stack(list(scores), dim=0).shape # returns [<generation_length>, <batch_size>, <vocab_size>], is the same for logits

- proccesses logits to compute correct scores, as this does not happen when generation_config.do_sample=True (-> leads to faster and potentially better generations)
- all off the scores are -inf in that case and can't really be used
- alas the following code will only approximate them and not get them correct completely (but close enough for valid results)

In [63]:

from transformers.generation.logits_process import RepetitionPenaltyLogitsProcessor, TemperatureLogitsWarper

processor = RepetitionPenaltyLogitsProcessor(1.2)
warper = TemperatureLogitsWarper(generation_config.temperature)

logits_shifted = torch.stack(list(logits), dim=0)[:-1, :, :].squeeze()
sequence_ids_shifted = sequence_ids.unsqueeze(0)[:, 1:]

logits_unshifted = torch.stack(list(logits), dim=0).squeeze()
sequence_ids_unshifted = sequence_ids.unsqueeze(0)[:, :]


scores_self = processor(input_ids, logits_unshifted)
print(scores_self.shape)
scores_self

torch.Size([312, 32768])


tensor([[-4.2200e+02,  4.8438e-01,  2.5625e+00,  ..., -5.5000e+00,
         -4.0000e+00, -3.5625e+00],
        [-3.0200e+02, -6.4062e+00, -3.1445e-01,  ..., -5.9688e+00,
         -4.7188e+00, -6.5312e+00],
        [-2.3900e+02, -8.5000e+00,  3.4531e+00,  ..., -7.2812e+00,
         -8.3125e+00, -6.4062e+00],
        ...,
        [-1.9300e+02, -9.8125e+00,  1.1562e+01,  ..., -8.8750e+00,
         -7.6250e+00, -6.6250e+00],
        [-2.4900e+02, -6.5625e+00,  1.4625e+01,  ..., -8.8125e+00,
         -7.8750e+00, -5.0000e+00],
        [-4.9800e+02, -3.0469e+00,  1.9500e+01,  ..., -5.4375e+00,
         -5.8438e+00, -5.8750e+00]])

In [17]:
i = 0
res = []
for row in logits_unshifted:
    # apply processor on each logit row instead of the whole logits at once -> leads to less rounding errors
    tmp = row.unsqueeze(0).clone().type(torch.FloatTensor)

    res.append(processor(input_ids, tmp))
    scores_self[i] = processor(input_ids, tmp) / generation_config.temperature
    i += 1

In [19]:
scores_self

tensor([[-5480000.0000,    16953.1250,    43125.0000,  ...,
           -44375.0000,   -43750.0000,   -45937.5000],
        [-3180000.0000,   -62187.5000,    11640.6250,  ...,
           -63125.0000,   -45312.5000,   -70937.5000],
        [-2560000.0000,   -80625.0000,    45000.0000,  ...,
           -71562.5000,   -84375.0000,   -67187.5000],
        ...,
        [-2860000.0000,   -73750.0000,    51875.0000,  ...,
           -47812.5000,   -57500.0000,   -60937.5000],
        [-2840000.0000,   -74062.5000,   105000.0000,  ...,
           -77500.0000,   -69062.5000,   -61875.0000],
        [-4820000.0000,   -23125.0000,   168750.0000,  ...,
           -64062.5000,   -61562.5000,   -51250.0000]])

In [65]:
vocab_size = 32768
scores_transition = scores_self.transpose(0, 1)
print(scores_transition.shape)
print(torch.stack(list(logits), dim=0).squeeze().shape)
scores_transition = scores_transition.reshape(-1, vocab_size, scores_transition.shape[-1])
print(scores_transition.shape)
scores_transition = torch.nn.functional.log_softmax(scores_transition, dim=1)

print(scores_transition.shape)
print(sequence_ids_shifted.shape)
print(sequences[:,input_ids.shape[1]:].shape)
indices = sequence_ids_shifted

# 8. Compute scores
transition_scores_self = scores_transition.squeeze().gather(0, indices)
transition_scores_self 

torch.Size([32768, 249])
torch.Size([249, 32768])
torch.Size([1, 32768, 249])
torch.Size([1, 32768, 249])
torch.Size([1, 248])
torch.Size([1, 249])


tensor([[ -9.5944, -12.4211, -10.8303, -10.2502,  -6.7545, -14.4436, -14.8937,
         -11.8751, -12.7679,  -8.3280, -12.4921, -14.8516, -12.2119, -14.9990,
         -14.1860, -12.9380, -17.8222, -12.1459,  -8.0014,  -8.8141, -22.2299,
          -4.8453, -12.5011, -10.9393, -14.9311, -14.2549, -15.1632,  -6.6746,
         -16.2790, -14.2631,  -4.3065, -13.6988,  -7.8448, -14.6946,  -7.8094,
         -10.1293, -16.7325, -15.6250, -27.8126, -20.9063,  -2.1949, -13.1855,
         -13.1616, -27.1836, -17.9704, -22.1720, -19.1727, -16.1875, -11.1255,
         -22.7657, -21.9532, -17.9376, -16.0168, -12.3128, -18.8283, -22.4396,
         -15.1592, -14.7189, -22.0314, -22.5156, -18.6406, -23.1250, -22.5247,
         -10.5006, -21.2423, -17.4688, -21.7188, -16.2507, -13.3658, -15.9087,
         -18.0314, -14.7500, -24.8750, -20.2813, -14.8928,  -5.4550, -12.6192,
         -10.7569, -14.1770, -15.8129,  -7.1367, -15.5044, -11.3808, -11.5004,
         -14.5989, -17.1250, -18.5330, -14.5287,  -5

In [64]:
import math

def compute_perplexity(log_probs):
    total_log_prob = 0
    for log_prob in log_probs:
        total_log_prob += log_prob
    perplexity = math.exp(-total_log_prob / len(log_probs))
    return perplexity

In [69]:
import numpy as np


transition_scores = model.compute_transition_scores(sequences, torch.unbind(scores_self.unsqueeze(1), dim=0), normalize_logits=True)

generated_tokens = sequences[-1][input_ids.shape[1]:][1:]


log_probs = transition_scores[0][:-1]


print("PERPLEXITY: ", compute_perplexity(log_probs))
 
for tok, score in zip(generated_tokens, log_probs):

    # | token | token string | logits | probability

    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

PERPLEXITY:  1.371638369396272
|  5140 | answer   | -0.168 | 84.49%
|  1224 | this     | -0.015 | 98.54%
|  3764 | question | -1.224 | 29.42%
| 29493 | ,        | -0.000 | 99.99%
|  1246 | we       | -0.003 | 99.70%
|  1695 | need     | -0.002 | 99.79%
|  1066 | to       | -0.012 | 98.85%
|  9819 | identify | -0.000 | 99.97%
|  1040 | the      | -0.464 | 62.85%
|  9064 | requirements | -0.038 | 96.28%
|  1210 | or       | -1.039 | 35.39%
|  1492 | pr       | -2.470 | 8.46%
|  1165 | ere      | -0.620 | 53.77%
| 11993 | quis     | -0.000 | 100.00%
|  4155 | ites     | -0.000 | 99.99%
|  7851 | mentioned | -0.000 | 99.99%
|  1065 | in       | -2.054 | 12.82%
|  1040 | the      | -0.063 | 93.91%
|  3526 | context  | -0.000 | 99.96%
|  1122 | for      | -0.079 | 92.40%
| 24256 | admission | -0.143 | 86.64%
|  1546 | into     | -1.099 | 33.31%
|  1040 | the      | -0.704 | 49.48%
| 10129 | Master   | -0.007 | 99.29%
| 29510 | '        | -0.454 | 63.51%
| 29481 | s        | -0.012 | 98.81%
|

## when do_sample=False it will work directly with the scores output without the preprocessing by hand

In [5]:
transition_scores = model.compute_transition_scores(sequences, scores, normalize_logits=True)
import numpy as np

generated_tokens = sequences[-1][input_ids.shape[1]:]

print("PERPLEXITY: ", compute_perplexity(transition_scores[0]))

for tok, score in zip(generated_tokens, transition_scores[0]):

    # | token | token string | logits | probability

    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

PERPLEXITY:  1.9461695291850882
|  1183 | The      | -3.542 | 2.89%
|  3526 | context  | -1.157 | 31.44%
|  6080 | provides | -0.329 | 71.98%
|  3624 | several  | -2.505 | 8.17%
|  3645 | options  | -0.345 | 70.85%
|  1137 | that     | -1.684 | 18.56%
|  1309 | can      | -0.010 | 98.98%
|  7799 | serve    | -1.920 | 14.66%
|  1158 | as       | -0.000 | 100.00%
|  7935 | proof    | -0.255 | 77.52%
|  1070 | of       | -0.013 | 98.75%
|  6335 | German   | -0.004 | 99.58%
|  4610 | language | -0.001 | 99.89%
|  7034 | skills   | -0.186 | 83.02%
| 29491 | .        | -0.338 | 71.34%
|  3725 | These    | -0.599 | 54.92%
|  3792 | include  | -0.038 | 96.32%
| 29515 | :        | -0.182 | 83.35%
|  1027 |          | -3.716 | 2.43%
|   781 | 
        | -0.212 | 80.88%
| 29508 | 1        | -1.278 | 27.85%
| 29491 | .        | -0.010 | 99.01%
|  1098 | A        | -0.080 | 92.27%
| 16511 | certificate | -0.474 | 62.23%
|  8870 | showing  | -2.526 | 8.00%
|  6821 | successful | -1.172 | 30.99%
| 15

In [116]:
torch.unbind(scores_self.unsqueeze(1), dim=0)

(tensor([[-548.0000,    1.6953,    3.5937,  ...,   -4.4375,   -4.3750,
            -4.5938]]),
 tensor([[-318.0000,   -6.2188,    1.1641,  ...,   -6.3125,   -4.5312,
            -7.0938]]),
 tensor([[-256.0000,   -8.0625,    4.5000,  ...,   -7.1562,   -8.4375,
            -6.7188]]),
 tensor([[-274.0000,   -7.8125,    5.8125,  ...,   -6.8750,   -7.9062,
            -7.7500]]),
 tensor([[-310.0000,   -7.1875,    3.8750,  ...,   -7.7812,   -7.6875,
            -7.4062]]),
 tensor([[-304.0000,   -7.7500,    3.7969,  ...,   -6.7812,   -8.4375,
            -7.5312]]),
 tensor([[-228.0000,   -9.2500,    4.1250,  ...,   -6.5000,   -8.8125,
            -7.9688]]),
 tensor([[-280.0000,   -9.0625,    5.5000,  ...,   -7.4062,   -9.1250,
            -9.6875]]),
 tensor([[-193.0000,   -8.1875,    3.1406,  ...,   -4.9062,   -6.5625,
            -7.3750]]),
 tensor([[-310.0000,   -6.9688,    4.4062,  ...,   -6.6875,   -6.7812,
            -6.2812]]),
 tensor([[-302.0000,   -6.7500,    2.2031,  ...,  

In [118]:
# simulates processors and gets next tokens


min(50, scores_self.size(-1))
top_k = 50
indices_to_remove = scores_self < torch.topk(scores_self, top_k)[0][..., -1, None]
indices_to_remove
scores_processed_top_k = scores_self.masked_fill(indices_to_remove, -float("Inf"))
torch.max(scores_processed_top_k [0])

top_p = 0.0

sorted_logits, sorted_indices = torch.sort(scores_processed_top_k, descending=False)
cumulative_probs = sorted_logits.softmax(dim=-1).cumsum(dim=-1)

# Remove tokens with cumulative top_p above the threshold (token with 0 are kept)
sorted_indices_to_remove = cumulative_probs <= (1 - top_p)
# Keep at least min_tokens_to_keep
sorted_indices_to_remove[..., -1 :] = 0

# scatter sorted tensors to original indexing
indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
scores_processed_top_p = scores_processed_top_k.masked_fill(indices_to_remove, -float("Inf"))
print(torch.max(scores_processed_top_k [0]))
scores_processed_top_p 

probs = F.softmax(scores_processed_top_p , dim=-1)
next_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)
next_tokens

AttributeError: 'tuple' object has no attribute 'shape'

In [67]:
def to_tokens_and_logprobs(sequence_ids, logits):
    probs = torch.log_softmax(torch.stack(list(logits), dim=0), dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:-1, :, :] # match shift -> no probability for first token
    sequence_ids = sequence_ids.unsqueeze(0)[:, 1:] # shift away first token as probs[0] -> token[1]
    gen_probs = torch.gather(probs, 2, sequence_ids[:,None,:]).squeeze(1)
    print(gen_probs.shape)
    print(sequence_ids.shape)
    batch = []
    for generation, probs in zip(sequence_ids, gen_probs):
        text_sequence = []
        #print(generation.shape)
        #print(probs.shape)
        for token, p in zip(generation, probs):
            if token not in tokenizer.all_special_ids:
                #print(token.shape)
                #print(p.shape)
                text_sequence.append((tokenizer.decode(token.item()), p.item()))
        batch.append(text_sequence)
    return batch

In [68]:
result = to_tokens_and_logprobs(sequences[-1][input_ids.shape[1]:], logits)
result

torch.Size([1, 248])
torch.Size([1, 248])


[[('answer', -8.649835586547852),
  ('this', -11.462335586547852),
  ('question', -15.505304336547852),
  (',', -15.126398086547852),
  ('we', -15.587335586547852),
  ('need', -13.923273086547852),
  ('to', -5.087335586547852),
  ('identify', -13.438898086547852),
  ('the', -11.321710586547852),
  ('documents', -16.55608558654785),
  ('that', -14.024835586547852),
  ('can', -15.407648086547852),
  ('be', -14.173273086547852),
  ('accepted', -12.829523086547852),
  ('as', -13.423273086547852),
  ('proof', -17.47796058654785),
  ('of', -13.173273086547852),
  ('German', -10.852960586547852),
  ('language', -14.243585586547852),
  ('skills', -17.72796058654785),
  ('mentioned', -14.237726211547852),
  ('in', -12.477960586547852),
  ('the', -11.321710586547852),
  ('context', -10.524835586547852),
  ('.', -13.079523086547852),
  ('The', -3.5873355865478516),
  ('relevant', -15.247491836547852),
  ('information', -16.03264808654785),
  ('is', -13.974054336547852),
  ('found', -16.5248355865

In [25]:
print(torch.stack(list(logits), dim=0).shape)
print(input_ids.shape)
print(sequences.shape)
sequences[-1][input_ids.shape[1]:].shape

torch.Size([647, 1, 32768])
torch.Size([1, 3163])
torch.Size([1, 3810])


torch.Size([647])